## 모듈로 불러다가 Feature값 추출하기 - 4초

In [1]:
from omegaconf import OmegaConf
from models.i3d.extract_i3d import ExtractI3D

# 설정을 프로그래밍 방식으로 생성합니다.
args = OmegaConf.create({
    'feature_type': 'i3d',
    'device': 'cuda:0',  # 또는 'cpu'를 사용하시면 됩니다.
    'on_extraction': 'ignore',  # 이 설정은 무시될 것입니다.
    'output_path': 'ignore',  # 이 설정도 무시됩니다.
    'stack_size': 64,  # 이 값들은 예시이며 실제 값으로 대체해야 합니다.
    'step_size': 64,
    'streams': None,
    'flow_type': 'raft',
    'extraction_fps': None,
    'tmp_path': './tmp/i3d',
    'keep_tmp_files': False,
    'show_pred': False,
    'config': None
    # 여기에 args_cli에 필요한 나머지 설정을 추가하십시오.
})

# Extractor 인스턴스를 만듭니다.
extractor = ExtractI3D(args)

# 추출 함수를 정의합니다.
def extract_features(video_path):
    # 영상으로부터 feature를 추출합니다.
    features = extractor.extract(video_path)
    # RGB features를 가져옵니다.
    rgb_features = features['rgb']
    return rgb_features

# 사용 예시입니다.
video_path = '../PreProcess/ori_Training/C_3_12_1_BU_SMB_08-28_16-25-27_CC_RGB_DF2_M1.mp4'  # 여기를 실제 비디오 파일 경로로 바꾸십시오.
rgb_features = extract_features(video_path)
print(rgb_features)

[[0.03268447 0.1038489  0.06566148 ... 0.43930167 0.45920089 0.04780752]
 [0.04922621 0.23027417 0.0851275  ... 0.3272149  0.50672305 0.00901898]]


## 직접 필요한거 추려서 주피터 셀별로 실행하기 - 2초

## 첫 번째 셀: 클래스와 필요한 라이브러리 정의

In [2]:
# 필요한 라이브러리와 모듈을 임포트합니다.
import os
import cv2
import numpy as np
import torch
import torchvision
from typing import Dict
from models.i3d.i3d_src.i3d_net import I3D  # I3D 모델 구현
from models._base.base_extractor import BaseExtractor  # 기본 특징 추출기 클래스를 상속
from models.transforms import (PILToTensor, ResizeImproved, ScaleTo1_1, TensorCenterCrop, ToFloat, PermuteAndUnsqueeze)
from utils.io import reencode_video_with_diff_fps  # 동영상 FPS 조정 유틸리티

# I3D 모델을 사용해 RGB 데이터만 추출하는 클래스
class ExtractI3D(BaseExtractor):
    def __init__(self, args) -> None:
        super().__init__(
            feature_type=args['feature_type'],
            on_extraction=args.get('on_extraction', 'save_numpy'),
            tmp_path=args['tmp_path'],
            output_path=args['output_path'],
            keep_tmp_files=args['keep_tmp_files'],
            device=args['device'],
        )
        self.streams = ['rgb']
        self.i3d_classes_num = 400
        self.min_side_size = 256
        self.central_crop_size = 224
        self.extraction_fps = args.get('extraction_fps', None)
        self.step_size = args.get('step_size', 64)
        self.stack_size = args.get('stack_size', 64)
        self.resize_transforms = torchvision.transforms.Compose([
            torchvision.transforms.ToPILImage(),
            ResizeImproved(self.min_side_size),
            PILToTensor(),
            ToFloat(),
        ])
        self.i3d_transforms = {
            'rgb': torchvision.transforms.Compose([
                TensorCenterCrop(self.central_crop_size),
                ScaleTo1_1(),
                PermuteAndUnsqueeze()
            ])
        }
        self.name2module = self.load_model()

    @torch.no_grad()
    def extract(self, video_path: str) -> Dict[str, np.ndarray]:
        if self.extraction_fps is not None:
            video_path = reencode_video_with_diff_fps(video_path, self.tmp_path, self.extraction_fps)

        cap = cv2.VideoCapture(video_path)
        rgb_stack = []
        feats_dict = {'rgb': []}

        first_frame = True
        while cap.isOpened():
            frame_exists, rgb = cap.read()

            if first_frame:
                first_frame = False
                if not frame_exists:
                    continue

            if frame_exists:
                rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
                rgb = self.resize_transforms(rgb)
                rgb = rgb.unsqueeze(0)
                rgb_stack.append(rgb)

                if len(rgb_stack) == self.stack_size + 1:
                    batch_feats_dict = self.run_on_a_stack(rgb_stack)
                    feats_dict['rgb'].extend(batch_feats_dict['rgb'].tolist())
                    rgb_stack = rgb_stack[self.step_size:]  # 스텝 사이즈만큼 스택에서 제거
            else:
                cap.release()
                break

        if (self.extraction_fps is not None) and (not self.keep_tmp_files):
            os.remove(video_path)

        feats_dict = {stream: np.array(feats) for stream, feats in feats_dict.items()}
        return feats_dict

    def run_on_a_stack(self, rgb_stack) -> Dict[str, torch.Tensor]:
        models = self.name2module['model']
        rgb_stack = torch.cat(rgb_stack[:-1]).to(self.device)  # 마지막 프레임 제거
        stream_slice = self.i3d_transforms['rgb'](rgb_stack)
        batch_feats_dict = {'rgb': models['rgb'](stream_slice, features=True)}
        return batch_feats_dict

    def load_model(self) -> Dict[str, torch.nn.Module]:
        i3d_weights_paths = {'rgb': './models/i3d/checkpoints/i3d_rgb.pt'}
        name2module = {}

        i3d_stream_model = I3D(num_classes=self.i3d_classes_num, modality='rgb')
        i3d_stream_model.load_state_dict(torch.load(i3d_weights_paths['rgb'], map_location='cpu'))
        i3d_stream_model = i3d_stream_model.to(self.device)
        i3d_stream_model.eval()
        name2module['model'] = {'rgb': i3d_stream_model}

        return name2module


## 두 번째 셀: 설정과 인스턴스 생성

In [3]:
# 필요한 설정을 담은 args 딕셔너리를 정의합니다.
# 이 설정은 ExtractI3D 클래스를 사용하기 위해 필요한 정보를 포함합니다.
# 실제 경로 및 설정은 사용 환경에 맞게 조정해야 합니다.
from omegaconf import OmegaConf

args = OmegaConf.create({
    'feature_type': 'i3d',
    'device': 'cuda:0',  # 또는 'cpu'를 사용하시면 됩니다.
    'on_extraction': 'ignore',  # 이 설정은 무시될 것입니다.
    'output_path': 'ignore',  # 이 설정도 무시됩니다.
    'stack_size': 64,  # 이 값들은 예시이며 실제 값으로 대체해야 합니다.
    'step_size': 64,
    'streams': None,
    'flow_type': 'raft',
    'extraction_fps': None,
    'tmp_path': './tmp/i3d',
    'keep_tmp_files': False,
    'show_pred': False,
    'config': None
    # 여기에 args_cli에 필요한 나머지 설정을 추가하십시오.
})

# ExtractI3D 클래스의 인스턴스를 생성합니다.
# 위에서 정의한 args 딕셔너리를 생성자에 전달합니다.
extractor = ExtractI3D(args)


## 세 번째 셀: 동영상 처리

In [4]:
# 동영상 파일 경로를 지정합니다.
# 이 경로는 실제로 특징을 추출하고자 하는 동영상 파일의 위치를 가리킵니다.
# 사용자 환경에 맞게 해당 경로를 수정해야 합니다.
video_path = '../PreProcess/ori_Training/C_3_12_1_BU_SMB_08-28_16-25-27_CC_RGB_DF2_M1.mp4'

# 앞서 생성한 ExtractI3D 인스턴스의 extract 메소드를 호출하여
# 지정된 동영상 파일에서 RGB 특징을 추출합니다.
# 이 메소드는 추출된 특징들을 사전 형태로 반환합니다.
features = extractor.extract(video_path)

# 추출된 RGB 특징을 numpy 배열로 받아서 변수에 저장합니다.
# 'rgb' 키를 사용하여 사전에서 RGB 특징에 접근할 수 있습니다.
rgb_features = features['rgb']

# 선택적: 추출된 특징의 크기나 내용을 확인하기 위해 print 함수를 사용할 수 있습니다.
print(f"추출된 RGB 특징의 크기: {rgb_features.shape}")
print(rgb_features)

추출된 RGB 특징의 크기: (2, 1024)
[[0.03268447 0.1038489  0.06566148 ... 0.43930167 0.45920089 0.04780752]
 [0.04922621 0.23027417 0.0851275  ... 0.3272149  0.50672305 0.00901898]]
